In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

print("🐺 HOLDINGS ANALYSIS")
print("="*70)
print("Checking Tyr's actual positions against market data")
print("="*70)

## YOUR POSITIONS (From Screenshots)

**Robinhood (~$838 total):**
- UUUU: 7 shares = $129.78 (~$18.54/share)
- USAR: 7 shares = $125.79 (~$17.97/share)  
- NVTS: 10 shares = $103.40 (~$10.34/share)
- ASTS: 1 share = $95.44
- SIDU: 15 shares = $78.75 (~$5.25/share)

**Fidelity (~$513 total):**
- MU: Unknown shares, +$10.42 today
- IONQ: Unknown shares, +$3.14 today
- UUUU: Unknown shares, +$0.05 today

In [ ]:
# Pull recent data for all holdings
holdings = {
    'UUUU': {'shares': 7, 'current_value': 129.78},
    'USAR': {'shares': 7, 'current_value': 125.79},
    'NVTS': {'shares': 10, 'current_value': 103.40},
    'ASTS': {'shares': 1, 'current_value': 95.44},
    'SIDU': {'shares': 15, 'current_value': 78.75},
}

# Pull 3 months of data
start_date = '2025-10-01'
end_date = '2026-01-07'

data = {}

print("\nPulling 3 months of data...")
print("-"*70)

for ticker in holdings.keys():
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        if not hist.empty:
            data[ticker] = hist
            print(f"✓ {ticker}: {len(hist)} days")
    except Exception as e:
        print(f"✗ {ticker}: {e}")

## SIDU - Let's Check Your Claim
You said SIDU "has been getting up." Let me see the actual price action.

In [ ]:
if 'SIDU' in data:
    sidu = data['SIDU']
    
    print("\n" + "="*70)
    print("SIDU PRICE HISTORY")
    print("="*70)
    
    # Key dates
    print("\nKey Prices:")
    print("-"*70)
    
    # October high
    oct_high = sidu[sidu.index < '2025-11-01']['High'].max()
    oct_high_date = sidu[sidu.index < '2025-11-01']['High'].idxmax()
    print(f"Oct High: ${oct_high:.2f} on {oct_high_date.strftime('%Y-%m-%d')}")
    
    # November high
    nov_high = sidu[(sidu.index >= '2025-11-01') & (sidu.index < '2025-12-01')]['High'].max()
    nov_high_date = sidu[(sidu.index >= '2025-11-01') & (sidu.index < '2025-12-01')]['High'].idxmax()
    print(f"Nov High: ${nov_high:.2f} on {nov_high_date.strftime('%Y-%m-%d')}")
    
    # December high
    if len(sidu[sidu.index >= '2025-12-01']) > 0:
        dec_high = sidu[sidu.index >= '2025-12-01']['High'].max()
        dec_high_date = sidu[sidu.index >= '2025-12-01']['High'].idxmax()
        print(f"Dec High: ${dec_high:.2f} on {dec_high_date.strftime('%Y-%m-%d')}")
    
    # Current price
    current = sidu['Close'].iloc[-1]
    current_date = sidu.index[-1]
    print(f"\nCurrent: ${current:.2f} on {current_date.strftime('%Y-%m-%d')}")
    
    # Your entry (estimated from Jan 6 area)
    jan6_area = sidu[sidu.index >= '2026-01-06']
    if len(jan6_area) > 0:
        jan6_open = jan6_area['Open'].iloc[0]
        jan6_high = jan6_area['High'].iloc[0]
        jan6_low = jan6_area['Low'].iloc[0]
        jan6_close = jan6_area['Close'].iloc[0]
        
        print(f"\nJan 6 Range: ${jan6_low:.2f} - ${jan6_high:.2f}")
        print(f"Jan 6 Close: ${jan6_close:.2f}")
        
        # Your cost basis (from current value / shares)
        your_cost = holdings['SIDU']['current_value'] / holdings['SIDU']['shares']
        print(f"\nYour Estimated Entry: ${your_cost:.2f}")
        
        # P&L
        pnl_pct = ((current - your_cost) / your_cost) * 100
        pnl_dollars = (current - your_cost) * holdings['SIDU']['shares']
        
        print(f"\nYour P&L:")
        print(f"  Entry: ${your_cost:.2f}")
        print(f"  Current: ${current:.2f}")
        print(f"  Change: {pnl_pct:+.1f}%")
        print(f"  $ P&L: ${pnl_dollars:+.2f}")
    
    # Last 10 days detail
    print("\n" + "="*70)
    print("LAST 10 DAYS - Day by Day")
    print("="*70)
    recent = sidu.tail(10)
    for date, row in recent.iterrows():
        daily_change = ((row['Close'] - row['Open']) / row['Open']) * 100
        print(f"{date.strftime('%Y-%m-%d')}: ${row['Close']:6.2f} ({daily_change:+5.1f}%) | Range: ${row['Low']:.2f}-${row['High']:.2f}")

## ALL HOLDINGS - Current Status

In [ ]:
print("\n" + "="*70)
print("ALL ROBINHOOD HOLDINGS - Analysis")
print("="*70)

for ticker, position in holdings.items():
    if ticker not in data:
        continue
    
    df = data[ticker]
    current_price = df['Close'].iloc[-1]
    
    # Your cost basis (from screenshot)
    your_cost = position['current_value'] / position['shares']
    
    # P&L
    pnl_pct = ((current_price - your_cost) / your_cost) * 100
    pnl_dollars = (current_price - your_cost) * position['shares']
    
    # 5-day move
    five_days_ago = df['Close'].iloc[-6] if len(df) >= 6 else df['Close'].iloc[0]
    five_day_move = ((current_price - five_days_ago) / five_days_ago) * 100
    
    # Recent high
    recent_high = df['High'].tail(20).max()
    pct_from_high = ((current_price - recent_high) / recent_high) * 100
    
    print(f"\n{ticker}:")
    print("-"*70)
    print(f"  Your Entry: ${your_cost:.2f}")
    print(f"  Current: ${current_price:.2f}")
    print(f"  Your P&L: {pnl_pct:+.1f}% (${pnl_dollars:+.2f})")
    print(f"  5-Day Move: {five_day_move:+.1f}%")
    print(f"  From 20-Day High: {pct_from_high:+.1f}%")

## THE VERDICT

Let's see what the numbers ACTUALLY say about each holding.

In [ ]:
print("\n" + "="*70)
print("VERDICT: KEEP, ADD, OR EXIT?")
print("="*70)

for ticker in holdings.keys():
    if ticker not in data:
        continue
        
    df = data[ticker]
    
    # Calculate RSI
    changes = df['Close'].diff().tail(14)
    gains = changes[changes > 0].sum()
    losses = abs(changes[changes < 0].sum())
    if losses > 0:
        rs = gains / losses
        rsi = 100 - (100 / (1 + rs))
    else:
        rsi = 100
    
    # Volume trend
    vol_recent = df['Volume'].tail(5).mean()
    vol_avg = df['Volume'].tail(20).mean()
    vol_trend = vol_recent / vol_avg if vol_avg > 0 else 1
    
    # Trend
    ma20 = df['Close'].tail(20).mean()
    current = df['Close'].iloc[-1]
    above_ma20 = current > ma20
    
    print(f"\n{ticker}:")
    print(f"  RSI: {rsi:.0f}")
    print(f"  Volume Trend: {vol_trend:.1f}x avg")
    print(f"  Above 20-MA: {'✓' if above_ma20 else '✗'} (${ma20:.2f})")
    
    # Decision
    if rsi > 70:
        print(f"  🔥 HOT - Consider trimming")
    elif rsi < 35:
        if above_ma20 and vol_trend > 0.8:
            print(f"  ❄️ COLD - Potential buy zone if quality")
        else:
            print(f"  ⚠️ WEAK - Watch for further breakdown")
    else:
        if above_ma20:
            print(f"  ✅ HEALTHY - Hold")
        else:
            print(f"  ⚠️ BELOW TREND - Monitor")

# 🐺 WHERE IS THE NEXT PARTY?

## Finding Day 0-1 Opportunities (Not Day 3 Chasers)

In [ ]:
import yfinance as yf
import pandas as pd

# OUR UNIVERSE
SECTORS = {
    'Quantum': ['IONQ', 'RGTI', 'QBTS', 'QUBT', 'ARQQ'],
    'Nuclear': ['UUUU', 'USAR', 'NXE', 'DNN', 'LEU'],
    'Space': ['RKLB', 'ASTS', 'LUNR', 'PL', 'SIDU'],
    'Semi': ['NVTS', 'BRSH', 'NXPI', 'SWKS', 'MRVL'],
    'AI_Infra': ['AVGO', 'ARM', 'NVDA', 'SMCI', 'PLTR']
}

print("🐺 NEXT PARTY SCANNER - Day 0-1 Opportunities")
print("="*80)
print("Looking for: Volume spike + Moderate move (3-8%) + Early stage (≤1 green day)")
print("="*80)

candidates = []

for sector, tickers in SECTORS.items():
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period='10d')
            
            if len(hist) < 5:
                continue
            
            last_5 = hist.tail(5)
            today = last_5.iloc[-1]
            
            # Calculate metrics
            today_change = ((today['Close'] - today['Open']) / today['Open']) * 100
            vol_ratio = today['Volume'] / last_5['Volume'].iloc[:-1].mean()
            
            # Was it quiet before?
            previous_3_days = last_5.iloc[-4:-1]['Close'].pct_change().mean() * 100
            was_quiet = abs(previous_3_days) < 2
            
            # Count consecutive green days
            consecutive_green = 0
            for i in range(len(last_5)-1, -1, -1):
                day_gain = ((last_5.iloc[i]['Close'] - last_5.iloc[i]['Open']) / last_5.iloc[i]['Open']) * 100
                if day_gain > 1:
                    consecutive_green += 1
                else:
                    break
            
            # DAY 0-1 CRITERIA
            is_early_breakout = vol_ratio > 2.0 and 3 < today_change < 8 and consecutive_green <= 1
            is_stealth = vol_ratio > 1.5 and 0 < today_change < 3 and was_quiet
            
            if is_early_breakout or is_stealth:
                day_label = "DAY 1" if consecutive_green == 1 else "DAY 0"
                if is_stealth:
                    day_label = "DAY 0 (stealth)"
                
                candidates.append({
                    'ticker': ticker,
                    'sector': sector,
                    'day': day_label,
                    'change': today_change,
                    'vol_ratio': vol_ratio,
                    'price': today['Close'],
                    'was_quiet': was_quiet
                })
                
        except Exception as e:
            pass

# Sort by volume ratio
candidates.sort(key=lambda x: x['vol_ratio'], reverse=True)

print(f"\nFOUND {len(candidates)} EARLY STAGE OPPORTUNITIES:")
print("-"*80)

if len(candidates) == 0:
    print("❌ NO DAY 0-1 PARTIES FOUND")
    print("   → Either we're late to everything, or market is consolidating")
else:
    for c in candidates:
        stealth = " 📊" if c['was_quiet'] else ""
        print(f"{c['ticker']:6} ({c['sector']:8}): {c['day']:15} | ${c['price']:6.2f} | {c['change']:+5.1f}% | Vol {c['vol_ratio']:.1f}x{stealth}")

print("\n" + "="*80)

In [ ]:
# SECTOR ROTATION HEAT MAP
print("SECTOR HEAT MAP - Where's the money flowing?")
print("="*80)

sector_scores = {}
for sector, tickers in SECTORS.items():
    total_vol_spike = 0
    count = 0
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period='5d')
            if len(hist) >= 2:
                today_vol = hist.iloc[-1]['Volume']
                avg_vol = hist.iloc[:-1]['Volume'].mean()
                vol_ratio = today_vol / avg_vol if avg_vol > 0 else 1
                total_vol_spike += vol_ratio
                count += 1
        except:
            pass
    
    if count > 0:
        sector_scores[sector] = total_vol_spike / count

# Sort by heat
sorted_sectors = sorted(sector_scores.items(), key=lambda x: x[1], reverse=True)

for sector, score in sorted_sectors:
    if score > 1.5:
        heat = "🔥🔥🔥 RAGING"
    elif score > 1.2:
        heat = "🔥🔥 WARMING UP"
    elif score > 1.0:
        heat = "🔥 HEATING"
    else:
        heat = "❄️ COLD"
    
    print(f"{sector:12} {heat:15} {score:.2f}x avg volume")

print("\n💡 ROTATION SIGNAL: Money flows from cold → hot sectors")
print("   Look for tickers in HOTTEST sector that are still Day 0-1")

## 🎯 TOMORROW'S WATCH LIST (Jan 8 Premarket)

**What to watch:**
1. **Day 0-1 tickers** from scan above → Check if they gap up for confirmation
2. **Hottest sector tickers** → All movers in that sector (sector-wide = real party)
3. **Your current positions** → Day 3 runners need exit signals

**Entry Rules:**
- ✅ ENTER: Day 1-2, volume >2x, sector confirmation, +3-6% move
- ⚠️ CAUTION: Day 3, volume fading, solo mover (no sector confirmation)
- ❌ AVOID: Day 4+, volume <1x, overextended (>15% in 1 day)

**Exit Rules:**
- 🟢 HOLD: Day 1-3, volume strong, sector still hot
- 🟡 TRIM: Day 3-4, taking profits on 50%, let rest run
- 🔴 EXIT: Day 4+, volume fading, AH showing weakness, first red candle

In [ ]:
# SPECIFIC TICKERS TO WATCH TOMORROW
print("="*80)
print("🎯 TOMORROW MORNING (Jan 8) - SPECIFIC ACTIONS")
print("="*80)

# Get the candidates from our scan
if len(candidates) > 0:
    print("\n📊 NEW OPPORTUNITIES (Watch premarket):")
    print("-"*80)
    for c in candidates[:3]:  # Top 3
        print(f"\n{c['ticker']} - {c['sector']} sector")
        print(f"  Status: {c['day']}")
        print(f"  Yesterday: +{c['change']:.1f}% on {c['vol_ratio']:.1f}x volume")
        print(f"  Watch for: Gap up 3-5% = Day 1 confirmation")
        print(f"  Action: If confirmed, enter on first pullback")

# Your current Day 3 runners
print("\n\n🔥 YOUR CURRENT POSITIONS (Day 3 - Decision Time):")
print("-"*80)

day3_positions = {
    'USAR': '+10.9% yesterday, continuing strong',
    'NVTS': '+12.6% yesterday, continuing strong', 
    'SIDU': '+24.7% yesterday, watch for exhaustion',
    'ASTS': '+7.3% yesterday, AH showing weakness',
}

for ticker, note in day3_positions.items():
    print(f"\n{ticker}:")
    print(f"  Note: {note}")
    print(f"  Watch: Premarket action")
    print(f"  If gaps up again: Trim 30-50%")
    print(f"  If gaps down: Consider full exit")
    print(f"  If flat: Hold and watch for volume")

print("\n" + "="*80)
print("🐺 THE GAME: Catch Day 1, Ride Day 2-3, Exit Day 4")
print("="*80)